In [1]:
import torch 
import torch as tch

from torch import nn
import torchaudio as ta
import torchaudio.transforms as tatx
import torchaudio.functional as tafn
from pathlib import Path
import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import Audio

from train_tools import find_lr, check_with_patient, UpdatingPlotlyLines, fit_step, evaluate, num_param
from torch.optim import Adam
from torch.nn import functional as nnfn

%load_ext autoreload
%autoreload 2 

In [109]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
       

    def forward(self, x): 
        inp_size = x.size()
        return nn.functional.max_pool1d(input=x,
                  kernel_size= (inp_size[-1]))

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, 16, stride=8), 
            #nn.ReLU(), 

            nn.Conv1d(32, 64, 16, stride=8), 
            #nn.ReLU(), 

            nn.Conv1d(64, 128, 8, stride=4), 
            #nn.ReLU(), 

            nn.Conv1d(128, 256, 4, stride=2), 
            #nn.ReLU(), 
            
           
            # GlobalMaxPool1d(),

 

            # nn.Flatten(),
            # nn.Linear(256, 64),
            # #nn.ReLU(), 

            # nn.Linear(64, 1),
            # nn.Sigmoid(),
        )
    
    def forward(self, x):
        return tch.squeeze(self.conv(x))

m = Model()
x = tch.rand(1, 1, 6400)
print(num_param(m))
m(x).shape

230368


torch.Size([256, 10])

In [104]:
import onnxruntime
import onnx
import torch
def save_as_onnx(model, sample_x, file='temp/temp.onnx'):
    onnx_program = torch.onnx.dynamo_export(model, sample_x)
    onnx_program.save(file)

    onnx_model = onnx.load(file)
    onnx.checker.check_model(onnx_model)    


class RunONNX: 
    def __init__(self, file='temp/temp.onnx'):
        self.ort_session = onnxruntime.InferenceSession(file, providers=['CPUExecutionProvider'])

        self.inputs = self.ort_session.get_inputs()
        self.input_names = [i.name for i in self.inputs]

    def __call__(self, *x):
        return self.ort_session.run(None, dict(zip(self.input_names, x)))



In [105]:
xnp = x.numpy()
save_as_onnx(m, x)
m_onnx = RunONNX()

/home/kawa/projects/car1/venv3.11/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [107]:
for i in range(200):
    m_onnx(xnp)

In [108]:
m_onnx(xnp)

[array(0.5024913, dtype=float32)]

In [80]:
for i in range(10):
    m_onnx(xnp)

In [ ]:
4

In [26]:
(6400/16000)*200

80.0

/home/kawa/projects/car1/venv3.11/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
